In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec, KeyedVectors

c:\Users\shour\anaconda3\envs\shourya\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\shour\anaconda3\envs\shourya\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\shour\anaconda3\envs\shourya\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('IPL_Match_Highlights_Commentary.csv')

In [3]:
df.head()

,Match_id,Team,Over_num,Commentary,batsman,score
0,4281444,RCB 1st Inns,0.5,"Nehra to Mandeep, FOUR, first boundary for Man...",Nehra to Mandeep,FOUR
1,4281444,RCB 1st Inns,1.0,"Nehra to Mandeep, FOUR, back-to-back boundarie...",Nehra to Mandeep,FOUR
2,4281444,RCB 1st Inns,10.0,"Henriques to Kedar Jadhav, FOUR, hit straight ...",Henriques to Kedar Jadhav,FOUR
3,4281444,RCB 1st Inns,10.4,"Nehra to Kedar Jadhav, FOUR, another full toss...",Nehra to Kedar Jadhav,FOUR
4,4281444,RCB 1st Inns,10.5,"Nehra to Kedar Jadhav, FOUR, four more. Jadhav...",Nehra to Kedar Jadhav,FOUR


In [4]:
df.shape

(11574, 6)

In [5]:
df['score'].value_counts()

score
FOUR      5548
SIX       2679
OUT       2419
ONE        557
NO RUN     175
TWO        158
WIDE        17
THREE       17
FIVE         4
Name: count, dtype: int64

In [6]:
def batsman_names(input_str):
    names_list = input_str.split(' ')
    idx = names_list.index('to')
    batsman = names_list[idx+1:]
    bat_name = str()
    for name in batsman:
        bat_name += name
    return bat_name

In [7]:
def bowler_names(input_str):
    names_list = input_str.split(' ')
    idx = names_list.index('to')
    bowler = names_list[:idx]
    bowl_name = str()
    for name in bowler:
        bowl_name+= name 
    return bowl_name  

In [8]:
df['Batsman'] = df['batsman'].apply(batsman_names, 1)
df['Bowler'] = df['batsman'].apply(bowler_names, 1)

In [9]:
df.drop(['batsman'], axis=1, inplace=True)

In [10]:
df['Commentary'] = df['Commentary'].str.lower()

In [11]:
df.head()

,Match_id,Team,Over_num,Commentary,score,Batsman,Bowler
0,4281444,RCB 1st Inns,0.5,"nehra to mandeep, four, first boundary for man...",FOUR,Mandeep,Nehra
1,4281444,RCB 1st Inns,1.0,"nehra to mandeep, four, back-to-back boundarie...",FOUR,Mandeep,Nehra
2,4281444,RCB 1st Inns,10.0,"henriques to kedar jadhav, four, hit straight ...",FOUR,KedarJadhav,Henriques
3,4281444,RCB 1st Inns,10.4,"nehra to kedar jadhav, four, another full toss...",FOUR,KedarJadhav,Nehra
4,4281444,RCB 1st Inns,10.5,"nehra to kedar jadhav, four, four more. jadhav...",FOUR,KedarJadhav,Nehra


In [12]:
df['Commentary']

0        nehra to mandeep, four, first boundary for man...
1        nehra to mandeep, four, back-to-back boundarie...
2        henriques to kedar jadhav, four, hit straight ...
3        nehra to kedar jadhav, four, another full toss...
4        nehra to kedar jadhav, four, four more. jadhav...
                               ...                        
11569    rabada to smith, four, who'd've thought the wo...
11570    harshal patel to jaiswal, six, short ball, and...
11571    ashwin to smith, four, a rare rank half-tracke...
11572    ashwin to jaiswal, 1 run, pushes this one hast...
11573    nortje to smith, out caught by hetmyer! that's...
Name: Commentary, Length: 11574, dtype: object

In [13]:
import string
puncs = string.punctuation

In [14]:
puncs

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
def return_text(text):
    return text.translate(str.maketrans(' ', ' ', puncs))

In [16]:
df['Commentary'] = df['Commentary'].apply(return_text, 1)

In [17]:
df['Commentary']

0        nehra to mandeep four first boundary for mande...
1        nehra to mandeep four backtoback boundaries to...
2        henriques to kedar jadhav four hit straight ba...
3        nehra to kedar jadhav four another full toss i...
4        nehra to kedar jadhav four four more jadhav st...
                               ...                        
11569    rabada to smith four whodve thought the worlds...
11570    harshal patel to jaiswal six short ball and ja...
11571    ashwin to smith four a rare rank halftracker a...
11572    ashwin to jaiswal 1 run pushes this one hastil...
11573    nortje to smith out caught by hetmyer thats a ...
Name: Commentary, Length: 11574, dtype: object

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def remove_stops(text):
    text = text.split(' ')
    words = []
    for word in text:
        if word in stopwords.words('english'):
            words.append('')
        else:
            words.append(word)
    output_str = words[:]
    return " ".join(output_str)

In [20]:
df['Commentary'] = df['Commentary'].apply(remove_stops)

In [21]:
import spacy

In [22]:
splits=  spacy.load('en_core_web_sm')

In [51]:
print(splits)

In [23]:
from nltk import PorterStemmer
ps = PorterStemmer()

In [24]:
def stemming(text):
    return " ".join([ps.stem(word) for word in text.split(' ')])

In [25]:
df['Commentary'] = df['Commentary'].apply(stemming)

In [26]:
df.head()

,Match_id,Team,Over_num,Commentary,score,Batsman,Bowler
0,4281444,RCB 1st Inns,0.5,nehra mandeep four first boundari mandeep r...,FOUR,Mandeep,Nehra
1,4281444,RCB 1st Inns,1.0,nehra mandeep four backtoback boundari end ...,FOUR,Mandeep,Nehra
2,4281444,RCB 1st Inns,10.0,henriqu kedar jadhav four hit straight back ...,FOUR,KedarJadhav,Henriques
3,4281444,RCB 1st Inns,10.4,nehra kedar jadhav four anoth full toss jadh...,FOUR,KedarJadhav,Nehra
4,4281444,RCB 1st Inns,10.5,nehra kedar jadhav four four jadhav start r...,FOUR,KedarJadhav,Nehra


In [27]:
cv = CountVectorizer(min_df=10)

In [28]:
bow = cv.fit_transform(df['Commentary'])

In [29]:
bow = bow.toarray()

In [30]:
cv.vocabulary_

{'nehra': 1294,
 'mandeep': 1173,
 'four': 776,
 'first': 732,
 'boundari': 288,
 'rcb': 1538,
 'full': 788,
 'pad': 1383,
 'need': 1290,
 'put': 1503,
 'away': 191,
 'pick': 1421,
 'dispatch': 566,
 'midwicket': 1222,
 'coupl': 460,
 'bounc': 286,
 'fenc': 709,
 'backtoback': 199,
 'end': 637,
 'tad': 1926,
 'short': 1702,
 'length': 1114,
 'width': 2132,
 'cut': 494,
 'didnt': 550,
 'tri': 2021,
 'hit': 923,
 'hard': 885,
 'place': 1431,
 'behind': 236,
 'point': 1447,
 'bhuvi': 246,
 'third': 1962,
 'man': 1171,
 'gave': 806,
 'chase': 373,
 'pretti': 1475,
 'quickli': 1510,
 'henriqu': 912,
 'kedar': 1027,
 'jadhav': 1001,
 'straight': 1856,
 'back': 196,
 'late': 1082,
 'get': 816,
 'hand': 876,
 'offcutt': 1329,
 'almost': 130,
 'fool': 762,
 'shimmi': 1697,
 'check': 376,
 'drive': 596,
 'middl': 1217,
 'alright': 134,
 'beat': 231,
 'midoff': 1219,
 'fielder': 715,
 'anoth': 152,
 'toss': 2002,
 'time': 1984,
 'spot': 1821,
 'deep': 519,
 'great': 850,
 'bowl': 290,
 'he': 896,

In [31]:
def counts_lens(text):
    text_list = text.split(' ')
    return len(text_list)
    

In [32]:
df['text_len'] = df['Commentary'].apply(counts_lens)

In [33]:
df['text_len'].min()

10

In [34]:
two_gram = CountVectorizer(ngram_range=(2,4), min_df=100)

In [35]:
bow2 = two_gram.fit_transform(df['Commentary'])

In [36]:
bow2 = bow2.toarray()

In [37]:
len(bow2[5])

157

In [38]:
tfidf = TfidfVectorizer()
vectors = tfidf.fit_transform(df['Commentary'])

In [39]:
vectors = vectors.toarray()

In [40]:
df['Commentary']

0        nehra  mandeep four first boundari  mandeep  r...
1        nehra  mandeep four backtoback boundari  end  ...
2        henriqu  kedar jadhav four hit straight back  ...
3        nehra  kedar jadhav four anoth full toss  jadh...
4        nehra  kedar jadhav four four  jadhav start  r...
                               ...                        
11569    rabada  smith four whodv thought  world best t...
11570    harshal patel  jaiswal six short ball  jaiswal...
11571    ashwin  smith four  rare rank halftrack  that ...
11572    ashwin  jaiswal 1 run push  one hastili   back...
11573    nortj  smith  caught  hetmyer that  fabul low ...
Name: Commentary, Length: 11574, dtype: object

In [41]:
wv = Word2Vec(window=10, min_count=10, workers=6)

In [42]:
corpus = df['Commentary'].to_list()

In [43]:
input_corpus = []
for sentence in corpus:
    input_corpus.append(sentence.split(' '))

In [44]:
input_corpus

[['nehra',
  '',
  'mandeep',
  'four',
  'first',
  'boundari',
  '',
  'mandeep',
  '',
  'rcb',
  'full',
  '',
  '',
  '',
  'pad',
  'need',
  '',
  '',
  'put',
  'away',
  '',
  'mandeep',
  '',
  '',
  '',
  'pick',
  '',
  '',
  '',
  'dispatch',
  '',
  '',
  'midwicket',
  'coupl',
  '',
  'bounc',
  '',
  '',
  '',
  'fenc'],
 ['nehra',
  '',
  'mandeep',
  'four',
  'backtoback',
  'boundari',
  '',
  'end',
  '',
  'first',
  '',
  '',
  'nehra',
  '',
  '',
  'tad',
  'short',
  '',
  '',
  'length',
  'mandeep',
  '',
  '',
  'width',
  '',
  'cut',
  '',
  '',
  'didnt',
  'tri',
  '',
  'hit',
  '',
  'hard',
  '',
  'place',
  '',
  'behind',
  'point',
  '',
  'bhuvi',
  '',
  'third',
  'man',
  'gave',
  '',
  '',
  'chase',
  'pretti',
  'quickli'],
 ['henriqu',
  '',
  'kedar',
  'jadhav',
  'four',
  'hit',
  'straight',
  'back',
  '',
  'henriqu',
  '',
  '',
  '',
  'late',
  '',
  'get',
  '',
  'hand',
  '',
  '',
  '',
  '',
  'offcutt',
  '',
  'almost',

In [45]:
wv.build_vocab(input_corpus)

In [46]:
wv.train(input_corpus, total_examples=wv.corpus_count, epochs=wv.epochs)

(1532665, 2840255)

In [47]:
wv.wv.index_to_key

['',
 'four',
 'ball',
 'get',
 'six',
 'short',
 'back',
 'leg',
 'length',
 'run',
 'away',
 'deliveri',
 'b',
 'outsid',
 'deep',
 'full',
 'time',
 'caught',
 'one',
 'c',
 'bat',
 'shot',
 'midwicket',
 'wide',
 'cover',
 'squar',
 'man',
 'pull',
 'well',
 'boundari',
 'fine',
 'pandya',
 'take',
 'stump',
 'bowl',
 'fenc',
 'longon',
 'edg',
 'hit',
 'yadav',
 'across',
 'go',
 'de',
 'good',
 'right',
 'goe',
 'look',
 'wicket',
 'third',
 'point',
 'end',
 'slower',
 'that',
 'fielder',
 'pant',
 'he',
 'past',
 'got',
 'rahul',
 'hand',
 'line',
 'straight',
 'narin',
 'play',
 'backward',
 'way',
 'went',
 'catch',
 'cut',
 'dhawan',
 'anoth',
 'sharma',
 'front',
 'make',
 'come',
 'pick',
 'hardik',
 'toss',
 'kohli',
 'insid',
 'clear',
 'rohit',
 'tri',
 'raina',
 'first',
 'drive',
 'watson',
 'russel',
 'miss',
 'bowler',
 'enough',
 'khan',
 'dhoni',
 'extra',
 'middl',
 'longoff',
 'shreya',
 'dive',
 'stand',
 'iyer',
 'rashid',
 'pitch',
 'warner',
 'loft',
 '1',
 

In [48]:
wv.wv['run']

array([-1.4188759 , -2.2791688 , -1.984495  , -1.7186825 ,  0.746487  ,
       -1.4374839 , -1.5888095 ,  1.5702561 , -0.08392279, -0.6167258 ,
       -0.5592068 ,  0.6725669 , -0.2742751 , -0.78641057,  0.43631396,
        0.7662012 ,  0.6128229 , -0.42547673,  0.26583737, -0.53332615,
       -0.2972807 ,  1.2973189 , -1.6406853 , -1.547561  ,  1.6143482 ,
       -1.0204842 , -1.6065232 ,  0.3781547 , -1.3593683 , -0.988764  ,
        2.0097144 ,  0.4435295 ,  1.7145247 , -0.06874588, -1.8098012 ,
        0.99851084, -0.61631453,  0.3143628 ,  0.46717706,  0.02414221,
        1.643672  ,  0.52615416,  0.14562777, -1.1760912 , -0.3941839 ,
       -0.0444941 , -0.4713597 ,  0.8480554 ,  1.7987776 ,  2.0246277 ,
        0.6414591 , -0.41443768,  1.3120611 ,  1.1487826 ,  0.24006566,
       -1.7887872 , -0.7735008 ,  0.35001647,  0.91377443,  0.64362055,
       -0.06268383,  1.2745827 ,  2.5497513 , -1.1337979 , -0.4024629 ,
        0.9089878 , -0.0607374 , -2.278162  ,  0.33235946, -1.11